# 15 LoopStack

In [1]:
# default_exp loopstack

In [2]:
# export
from forgebox.loop import Loop,ProgressBar,Tolerate,Event
from types import MethodType

In [3]:
# export
def create_event(event_name):
    class BatchEvent(Event):pass
    BatchEvent.__name__ = event_name
    return BatchEvent

def events(*enames):
    return list(map(create_event,enames))

In [4]:
# export
class LoopStack(Loop):
    """
    A stack of loop
    """
    @classmethod
    def from_loops(cls,*loops):
        def init(self,iterable=[],name = None):
            name = name if name!=None else cls.__name__
            
            self.loops = dict()
            l = Loop(iterable)
            for L in loops:
                l = L(iterable = l)
            super().__init__(iterable = l,
                             name = name)
            
        setattr(cls,"__init__",init)
        return cls
    
    def __repr__(self,):
        return f"LoopStack>:{self.name}\n\t"+\
            "\n\t".join(map(str,self.core.layers[:-1]))

In [5]:
TRAIN_EVENTS = ["DATA_PROCESS","FORWARD","LOSS_CALC",
                "BACKWARD","OPT_STEP","METRICS"]
EVAL_EVENTS = ["DATA_PROCESS","EVAL_WRAP","FORWARD","LOSS_CALC","METRICS"]

In [13]:
# export 
class TrainLoop(LoopStack):
    def __init__(self,data_iter):
        self.from_loops(ProgressBar,Tolerate,
                        *events(*TRAIN_EVENTS))
        self.__init__(data_iter,)
        
        @self.on_DATA_PROCESS
        def opt_zero_grad(self):
            self.opt.zero_grad()
        
        @self.before_1st_FORWARD
        def switch_model_to_train(self):
            self.core.model = self.core.model.train()
            
class EvalLoop(LoopStack):
    def __init__(self,data_iter):
        self.from_loops(ProgressBar,Tolerate,
                        *events(*EVAL_EVENTS))
        self.__init__(data_iter,)
        
        @self.EVAL_FORWARD.downstream
        def torch_eval_wrap(self,func):
            with torch.no_grad():
                func()

In [14]:
loop = TrainLoop(range(50))
loop

LoopStack>:TrainLoop
	layer🍰Loop
	layer🍰ProgressBar
	layer🍰Tolerate
	event🌏DATA_PROCESS
	event🌏FORWARD
	event🌏LOSS_CALC
	event🌏BACKWARD
	event🌏OPT_STEP
	event🌏METRICS

In [8]:
loop.summary()

Loop layer TrainLoop summary:
🍰layer0.0	layer🍰Loop
--------------------------------------------------
🍰layer1.0	layer🍰ProgressBar
	[🐍func_name]	pgbar_data
	[⛰doc]	
        update progress bar with python dictionary
        data: python dictionary
        
	[😝var]	self,data
	[😜names]	t,set_postfix
	...................................
	[🐍func_name]	pgbar_description
	[⛰doc]	
        update progress bar prefix with text string
        
	[😝var]	self,text
	[😜names]	t,set_description_str
	...................................
--------------------------------------------------
🍰layer2.0	layer🍰Tolerate
	[🐍func_name]	error_list
	[⛰doc]	
        A list of errors happend so far
        
	[😝var]	self
	[😜names]	errors
	...................................
--------------------------------------------------
🍰layer3.0	event🌏DATA_PROCESS
	[🐍func_name]	after_last_DATA_PROCESS
	[⛰doc]	
            Append new after_last callback for event:DATA_PROCESS
            Use this function as decorator
        
	[😝va

In [9]:
from time import sleep

In [10]:
@loop.on_DATA_PROCESS
def process_data(loop):
    sleep(.1)
    loop.core.double = loop.element*2
    
@loop.on_FORWARD
def update_pg(loop):
    loop.pgbar_description(f"double: {loop.double}")
    
@loop.on_FORWARD
def another_thing_on_forward(loop):
    loop.pgbar_data(dict(a=1))

In [11]:
loop.run()